In [2]:
import os
import numpy as np
import matplotlib
import torch
# !pip install mat73
# import mat73
import matplotlib.pyplot as plt
import csv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split,TensorDataset
from torchvision import transforms, utils
import time
import pandas as pd
import scipy.io
import sklearn.metrics
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)

from scipy import signal

from sklearn.metrics import confusion_matrix

import seaborn as sn
import pandas as pd

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import LeaveOneGroupOut






In [3]:
from math import sqrt
import torch
import torch.nn.functional as F
from torch import nn
!pip install einops
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class LSA(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        self.heads = heads
        self.temperature = nn.Parameter(torch.log(torch.tensor(dim_head ** -0.5)))

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.temperature.exp()

        mask = torch.eye(dots.shape[-1], device = dots.device, dtype = torch.bool)
        mask_value = -torch.finfo(dots.dtype).max
        dots = dots.masked_fill(mask, mask_value)

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, LSA(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class SPT(nn.Module):
    def __init__(self, *, dim, patch_size, channels = 3):
        super().__init__()
        patch_dim = patch_size * patch_size * 5 * channels

        self.to_patch_tokens = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim)
        )

    def forward(self, x):
        shifts = ((1, -1, 0, 0), (-1, 1, 0, 0), (0, 0, 1, -1), (0, 0, -1, 1))
        shifted_x = list(map(lambda shift: F.pad(x, shift), shifts))
        x_with_shifts = torch.cat((x, *shifted_x), dim = 1)
        return self.to_patch_tokens(x_with_shifts)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = SPT(dim = dim, patch_size = patch_size, channels = channels)

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

In [4]:
def Load_TransModel(FeatType,person,opt):


  dataPath="/content/drive/My Drive/Colab Notebooks/RFMG/data/"
  FeatName=['stft','stft2','cwt','cwt2','cwt3'] 
  
  PATH=dataPath+"/model_all_vit/"+opt['modelVer']+"/"+FeatName[FeatType]+"_ex_"+str(person)+".pt"
  model_trans = ViT(
        image_size =opt['image_size'],
        patch_size = opt['patch_size'],
        num_classes =num_class,
        dim = par_vit['dim'],
        depth = par_vit['depth'],
        heads = par_vit['heads'],
        mlp_dim = par_vit['mlp_dim'],
        dropout = par_vit['dropout'],
        emb_dropout = par_vit['emb_dropout'],
        channels = in_ch
        ).cuda()
  model_trans.load_state_dict(torch.load(PATH))
  model_trans.eval()

  # for param in model_trans.parameters():
  #     param.requires_grad = False

  # model_trans.fc1 = nn.Linear(model_trans.fc1.in_features, model_trans.fc2.in_features)
  # model_trans.fc2 = nn.Linear(model_trans.fc2.in_features, num_class)
  return model_trans





In [5]:
in_ch=48
num_class=23
self_channel=[0,1,10,11,20,21,30,31]
par_vit1={'dim':512,'depth':6,'heads':16,'mlp_dim':32,'dropout':0.1,'emb_dropout':0.1};
par_vit2={'dim':1024,'depth':6,'heads':16,'mlp_dim':32,'dropout':0.1,'emb_dropout':0.1};
par_vit=par_vit1
parameter_vit=np.array(list(par_vit.items()));

def test_trans(X_all,y_all,opt,k_split,n_sel,trans_net):
  import torch
  torch.cuda.empty_cache()

  if __name__ == '__main__':
      
      transform = ToTensor()
      batchsize_train = 16

      folds=RepeatedStratifiedKFold(n_splits=k_split, n_repeats=1,random_state=100).split(X_all,y_all)

      test_ind_all=[]
      train_ind_all=[]
      for train_index, test_index in folds:
        test_ind_all.append(test_index)
        train_ind_all.append(train_index)
      

      # train_ind=train_ind_all[n_sel]
      # test_ind=test_ind_all[n_sel]

      # swap train & test index 
      # select one fold to be trained by n_sel
      train_ind=test_ind_all[n_sel]
      test_ind=train_ind_all[n_sel]


      X_train=X_all[train_ind] 
      X_test=X_all[test_ind]
      y_train=y_all[train_ind] 
      y_test=y_all[test_ind]

      train_loader = DataLoader(TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train)), batch_size=batchsize_train,shuffle=True)
      batchsize_test = 16
      test_loader = DataLoader(TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test)), batch_size=batchsize_test, shuffle=False)


      # (unique, counts) = np.unique(perNum_all_per[ind], return_counts=True)
      # frequencies = np.asarray((unique, counts)).T
      # print(frequencies)

      # (unique, counts) = np.unique(label_all_per[ind][train_ind], return_counts=True)
      # frequencies = np.asarray((unique, counts)).T
      # print(frequencies)
      # (unique, counts) = np.unique(label_all_per[ind][test_ind], return_counts=True)
      # frequencies = np.asarray((unique, counts)).T
      # print(frequencies)

      #hyperparameter definition   
      momentum = 0.1
      random_seed=1
      torch.backends.cudnn.enabled = False
      torch.manual_seed(random_seed)     #设定随机数种子为固定值
      
      train_loss_epoch = []
      test_acc_epoch =[]
      total_acc_epoch = []
      #epoch_range = np.arange(25,225,25)
      epoch_range = np.array([2])

   
      if  opt['trans']==0:
        network = ViT(
        image_size = opt['image_size'],
        patch_size = opt['patch_size'],
        num_classes =num_class,
        dim = par_vit['dim'],
        depth = par_vit['depth'],
        heads = par_vit['heads'],
        mlp_dim = par_vit['mlp_dim'],
        dropout = par_vit['dropout'],
        emb_dropout = par_vit['emb_dropout'],
        channels = in_ch
        ).cuda()
      if opt['trans']==1:
        network =trans_net.cuda()
      #training
      learning_rate=opt['learning rate']
      optimizer = optim.Adam(network.parameters(), lr=learning_rate)
      #optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)
      

      #network.train()
      Training_Loss = []
      Test_Loss = []
      start_time = time.time()
      criterion = nn.CrossEntropyLoss()
      #criterion = nn.CrossEntropyLoss(weight=torch.Tensor([1, 3]).cuda())
      for epoch in range(opt['n_epoch']):   # loop over the dataset multiple times
          train_loss = 0
          for X, Y in train_loader:
              # X = X.view(-1,1,X.shape[2],X.shape[3]).cuda()
              X = X.float().cuda()  
              Y = Y.long().view(-1, ).cuda() 
              current_batchsize = X.shape[0]
              optimizer.zero_grad()
              output = network(X)
              loss = criterion(output,Y)
              train_loss = train_loss + loss.item()
              loss.backward()                     #calculate the gradient decent
              optimizer.step()                    #update the weight
              
            
          test_loss = 0
          correct = 0
          total = 0
          test_y= []
          test_y_p = []
     
          with torch.no_grad():        
              # X = X.view(-1,X.shape[1],X.shape[2])
              # X = X.float()         
              for X, Y in test_loader:
                  X = torch.Tensor(X).cuda()
                  Y = torch.Tensor(Y).long().view(-1, ).cuda()
                  images, labels = X, Y
                  # calculate outputs by running images through the network
                  outputs = network(images)
                  loss = criterion(outputs,Y)
                  test_loss = test_loss + loss.item()

                  # the class with the highest energy is what we choose as prediction
                  _, predicted = torch.max(outputs.data, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum().item()
                  
                  for i in range(len(labels)):
                    test_y.append(labels[i])
                    test_y_p.append(predicted[i])
          
          #print('Accuracy of test cases: %d %%' % (100 * correct / total))
  

          Training_Loss.append(train_loss/len(train_loader.dataset))
          Test_Loss.append(test_loss/len(test_loader.dataset))
          # if epoch%2==0:
          #     print(train_loss/len(train_loader.dataset))
          #     print(test_loss/len(test_loader.dataset))
        
      
      # train_loss_epoch.append(Training_Loss[-1])
      # training_time=time.time()-start_time
      #print('total training time is',training_time)
      
      # plt.figure()
      # plt.plot(Training_Loss)
      # plt.title("training loss")
      # plt.xlabel("epoch")
      # plt.show()

      # plt.figure()
      # plt.plot(Test_Loss)
      # plt.title("test loss")
      # plt.xlabel("epoch")
      # plt.plot(Training_Loss)
      # plt.show()

  test_y_p=torch.FloatTensor(test_y_p)
  test_y=torch.FloatTensor(test_y)
  test_y_p=np.array(test_y_p.cpu())
  test_y=np.array(test_y.cpu())
  test_y2=np.stack((test_y, test_y_p))
  
  train_ind_inAll=train_ind
  test_ind_inAll=test_ind

  trn_tst_ind=[train_ind_inAll,test_ind_inAll]

  cm=confusion_matrix(test_y, test_y_p)
  acc=accuracy_score(test_y, test_y_p)
  print(cm,acc)
  return cm,acc,test_y2,train_ind_inAll,test_ind_inAll





In [14]:
mVer='v1_per8'
opt1={'image_size':56,'patch_size':7,'modelVer':mVer}
opt2={'image_size':40,'patch_size':5,'modelVer':mVer}
opt3={'image_size':125,'patch_size':5,'modelVer':mVer}
## for testing kfold CV transfer option
n1=40
lr1=2e-4
# was lr1=1e-5 for multiple participant 8per dataset 
opt1_t1={'image_size':56,'patch_size':7,'trans':1,'learning rate':lr1,'n_epoch':n1}
opt2_t1={'image_size':40,'patch_size':5,'trans':1,'learning rate':lr1,'n_epoch':n1}
opt3_t1={'image_size':125,'patch_size':5,'trans':1,'learning rate':lr1,'n_epoch':n1}
## for testing kfold CV not transfer option
n2=20
opt1_t0={'image_size':56,'patch_size':7,'trans':0,'learning rate':1e-4,'n_epoch':n2}
opt2_t0={'image_size':40,'patch_size':5,'trans':0,'learning rate':1e-4,'n_epoch':n2}
opt3_t0={'image_size':125,'patch_size':5,'trans':0,'learning rate':1e-4,'n_epoch':n2}
opt_list=[opt1,opt2,opt3,opt3,opt3]
opt_t1_list=[opt1_t1,opt2_t1,opt3_t1,opt3_t1,opt3_t1]
opt_t0_list=[opt1_t0,opt2_t0,opt3_t0,opt3_t0,opt3_t0]

In [ ]:

# use one feature ver to test 
# (p,k_split,transOpt) p participant number 1~8
# k_split, one 
from google.colab import drive
drive.mount('/content/drive')


def main_trans_CV1(p_test,k_split,transOpt,FeatOpt,ExpVer):  

    

       

    # ExpVer_list=['2_24']  # trans to small variation in sensor position 
    # ExpVer_list=['2_1','2_16','2_19'] # sensor position vaiation
    file_name_data=r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/data_all_multi6_filt_0.1_5.npz"
    data=np.load(file_name_data)
    feature_2d_stft=data['feature_2d_stft']
    feature_2d_stft2=data['feature_2d_stft2']
    feature_2d_cwt=data['feature_2d_cwt']
    feature_2d_cwt2=data['feature_2d_cwt2']
    feature_2d_cwt3=data['feature_2d_cwt3']

    label_all=data['label_all']
    caseNum_all=data['caseNum_all']
    channel=np.linspace(0,47,48).astype(int) 
    in_ch=48
    num_class=23
   
    cm=[]
    cm_norm=[]
    test_y_pred=[]
    test_y_true=[]
    acc=[]
    acc_all=[]
    trn_ind=[]
    tst_ind=[]

    feature_list=[feature_2d_stft,feature_2d_stft2[:,:,:40,:40],feature_2d_cwt,feature_2d_cwt2,feature_2d_cwt3]
    feature_listName=["stft","stft2","cwt","cwt2","cwt3"]
    if __name__ == '__main__':
      
        l=FeatOpt  # feature list index. "stft","stft2","cwt","cwt2","cwt3"
  
    

       # k_split=3 # k-fold  1/n train 
       # n_sel=0 

        #model_trans=Load_TransModel(l,p_test,opt_list[l]) #  (FeatType,person,opt):     FeatType: ['stft','stft2','cwt','cwt2','cwt3'] 

        for j in range(k_split):
          if transOpt==1:
            opt_temp=opt_t1_list[l]
          if transOpt==0:
            opt_temp=opt_t0_list[l]
          opt_temp_arr=np.array(list(opt_temp.items()));

          model_trans=Load_TransModel(l,p_test,opt_list[l])
          cm_temp,acc_temp,test_y_temp,trn_ind_temp,tst_ind_temp=test_trans(feature_list[l][:,channel,:,:],label_all[:],opt_temp,k_split,j,model_trans)
  
    
          cm.append(cm_temp)
          acc_all.append(acc_temp)
          
          trn_ind.append(trn_ind_temp)
          tst_ind.append(tst_ind_temp)

      

          for k in range(len(tst_ind_temp)):
            test_y_pred.append(test_y_temp[1,k])
            test_y_true.append(test_y_temp[0,k])
  
        acc_all=np.array(acc_all)
        test_y_pred=np.array(test_y_pred)
        test_y_true=np.array(test_y_true)
        cm=confusion_matrix(test_y_pred, test_y_true)
        cm_norm=confusion_matrix(test_y_pred, test_y_true,normalize='true')
        acc=accuracy_score(test_y_pred, test_y_true)

   
    SaveName='ch'+str(in_ch)+'_'+str(k_split)+'fold_'+feature_listName[l]+'_trans_'+str(transOpt)
  
    print(cm_norm.shape)
    print(test_y_pred.shape)
    plotfigFullClass(cm_norm,acc,SaveName,ExpVer)
 
    np.savez_compressed("/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVer+"/result/npyFile/"+'vit_'+SaveName+".npz",\
                    cm_norm=cm_norm,cm=cm,acc=acc,acc_all=acc_all,\
                    test_y=np.stack((test_y_pred, test_y_true)),test_ind=np.array(tst_ind),\
                   parameter_vit=parameter_vit,opt=opt_temp_arr)
    
# main_trans_CV1(p_test,k_split,transOpt,FeatOpt)  


p_test=2 # general mode choose exc_#  not important differnece

ExpVer_list=['2_1','2_16','2_19'] # sensor position vaiation
for i in range(len(ExpVer_list)):
  for vv in range(5): # feat version 0~4
    for k in [4,5]:
      main_trans_CV1(p_test,k,1,vv,ExpVer_list[i])
    


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[59  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0 32  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3 31  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 21  5  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1 26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 25  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  4 18  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0 20  3  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  1  0  0  0  1 23  0  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 22  4  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  1  0 26  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 19  1  0  0  0  2  0  0  0  0  2  3]
 [ 0  0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[[60  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 32  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1 32  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 21  4  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 24  1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 24  2  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 24  3  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0 25  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 20  1  0  0  0  3  0  0  0  0  2  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 26  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 25  0  0  0  0  0  0  0

In [10]:


def plotfigFullClass(cm,acc,tle,ExpVerN):

    parent_dir = r"/content/drive/My Drive/Colab Notebooks/RFMG/data/"+ExpVerN+"/result/"
    path1 = os.path.join(parent_dir, 'cm_fig')
    os.makedirs(path1, exist_ok = True) 
    path1 = os.path.join(parent_dir, 'npyFile')
    os.makedirs(path1, exist_ok = True) 

    th=0.01

    labelNum=str(1)  #labelling method 1,2,3,4
    size=15
    name1=["R", "G","G*2","P1","P1*2","P2","P2*2","P23","P23*2","P4","P4*2","sG","sF","sP1","sP2","sP23","sP4","U","U*2","D","D*2","sU","sD"]
    name2=name1
    w=20 #fig size 1      change when label num change
    h=22   #fig size 2
    
    # true_num=0
    # for i in range (len(cm)):
    #   true_num=true_num+ cm[i,i]
    # acc=true_num/np.sum(cm)   
    
    df_cm = pd.DataFrame(cm, index=name1, columns=name2)
    plt.rcParams["font.family"] = "Times New Roman"
    plt.figure()
    sn.set(font_scale=1.2)
    mask = np.zeros_like(df_cm)
    mask[np.where(cm<th)] = True
    
    sn.heatmap(df_cm, cmap="YlGnBu",fmt=".2f",vmin=0, vmax=1.0,annot=True,mask=mask,square=True,cbar=False,annot_kws={"size": size,'fontsize':15})
 
    s01=' Acc={n:.3f}'.format(n=acc)
    #plt.text(a,b,s01,fontsize=size)
    
    tleSave='vit_'+tle
    plt.title(tle+s01)

    figure = plt.gcf()
    figure.set_size_inches(w, h)
    
    plt.savefig(parent_dir+r"cm_fig/"+tleSave+"CM.png", dpi=300)


